# Imports

In [89]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models,evaluation
from transformers import BertTokenizer
from csv import QUOTE_NONE
from torch.utils.data import DataLoader,Dataset, TensorDataset
import pandas as pd
import torch
from torch.nn.utils.rnn import pad_sequence
import pickle
import os


# Loading the data and explore it

In [14]:
multinli_train = pd.read_json("multinli_1.0/multinli_1.0_train.jsonl", lines=True)
multinli_test = pd.read_json("multinli_1.0/multinli_1.0_dev_matched.jsonl", lines=True)
# multinli_mismatched = pd.read_json("multinli_1.0/multinli_1.0_dev_mismatched.jsonl", lines=True)

In [9]:
multinli_train.head(5)

,annotator_labels,genre,gold_label,pairID,promptID,sentence1,sentence1_binary_parse,sentence1_parse,sentence2,sentence2_binary_parse,sentence2_parse
0,[neutral],government,neutral,31193n,31193,Conceptually cream skimming has two basic dime...,( ( Conceptually ( cream skimming ) ) ( ( has ...,(ROOT (S (NP (JJ Conceptually) (NN cream) (NN ...,Product and geography are what make cream skim...,( ( ( Product and ) geography ) ( ( are ( what...,(ROOT (S (NP (NN Product) (CC and) (NN geograp...
1,[entailment],telephone,entailment,101457e,101457,you know during the season and i guess at at y...,( you ( ( know ( during ( ( ( the season ) and...,(ROOT (S (NP (PRP you)) (VP (VBP know) (PP (IN...,You lose the things to the following level if ...,( You ( ( ( ( lose ( the things ) ) ( to ( the...,(ROOT (S (NP (PRP You)) (VP (VBP lose) (NP (DT...
2,[entailment],fiction,entailment,134793e,134793,One of our number will carry out your instruct...,( ( One ( of ( our number ) ) ) ( ( will ( ( (...,(ROOT (S (NP (NP (CD One)) (PP (IN of) (NP (PR...,A member of my team will execute your orders w...,( ( ( A member ) ( of ( my team ) ) ) ( ( will...,(ROOT (S (NP (NP (DT A) (NN member)) (PP (IN o...
3,[entailment],fiction,entailment,37397e,37397,How do you know? All this is their information...,( ( How ( ( ( do you ) know ) ? ) ) ( ( All th...,(ROOT (S (SBARQ (WHADVP (WRB How)) (SQ (VBP do...,This information belongs to them.,( ( This information ) ( ( belongs ( to them )...,(ROOT (S (NP (DT This) (NN information)) (VP (...
4,[neutral],telephone,neutral,50563n,50563,yeah i tell you what though if you go price so...,( yeah ( i ( ( tell you ) ( what ( ( though ( ...,(ROOT (S (VP (VB yeah) (S (NP (FW i)) (VP (VB ...,The tennis shoes have a range of prices.,( ( The ( tennis shoes ) ) ( ( have ( ( a rang...,(ROOT (S (NP (DT The) (NN tennis) (NNS shoes))...


In [15]:
multinli_train_reduced = pd.concat([multinli_train[i] for i in ["gold_label", "sentence1","sentence2"]], axis=1)
multinli_test_reduced = pd.concat([multinli_test[i] for i in ["gold_label", "sentence1","sentence2"]], axis=1)
multinli_train_reduced.head(5)

,gold_label,sentence1,sentence2
0,neutral,Conceptually cream skimming has two basic dime...,Product and geography are what make cream skim...
1,entailment,you know during the season and i guess at at y...,You lose the things to the following level if ...
2,entailment,One of our number will carry out your instruct...,A member of my team will execute your orders w...
3,entailment,How do you know? All this is their information...,This information belongs to them.
4,neutral,yeah i tell you what though if you go price so...,The tennis shoes have a range of prices.


In [23]:
print(multinli_train_reduced['gold_label'].unique())
print(multinli_test_reduced['gold_label'].unique())
print(multinli_test_reduced.shape)

['neutral' 'entailment' 'contradiction']
['neutral' 'contradiction' 'entailment' '-']
(10000, 3)


In [26]:
multinli_test_reduced= multinli_test_reduced.loc[multinli_test_reduced["gold_label"] != '-']
multinli_test_reduced.shape

(9815, 3)

In [16]:
multinli_test_reduced.head(5)

,gold_label,sentence1,sentence2
0,neutral,The new rights are nice enough,Everyone really likes the newest benefits
1,contradiction,This site includes a list of all award winners...,The Government Executive articles housed on th...
2,entailment,uh i don't know i i have mixed emotions about ...,"I like him for the most part, but would still ..."
3,contradiction,yeah i i think my favorite restaurant is alway...,My favorite restaurants are always at least a ...
4,contradiction,i don't know um do you do a lot of camping,I know exactly.


# Trying training the BERT in pytorch (time consuming)

In [ ]:
'''
This class is adapted from https://towardsdatascience.com/fine-tuning-pre-trained-transformer-models-for-sentence-entailment-d87caf9ec9db
This step is done to tokenize the dataset such as [CLS],[SEP],etc. In addition, marking the position of each sentence. (WE ARE INTERESTED TO GET THE SENTENCE EMBEDDING FIRST -->
then)
'''
# class MNLIDataBert(Dataset):

#   def __init__(self, train_df, val_df):
#     self.label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}

#     self.train_df = train_df
#     self.val_df = val_df

#     self.base_path = '/multinli_1.0/'
#     self.tokenizer = BertTokenizer.from_pretrained('bert-base-uncased', do_lower_case=True) # Using a pre-trained BERT tokenizer to encode sentences
#     self.train_data = None
#     self.val_data = None
#     self.init_data()

#   def init_data(self):
#     self.train_data = self.load_data(self.train_df)
#     self.val_data = self.load_data(self.val_df)

#   def load_data(self, df):
#     MAX_LEN = 512
#     token_ids = []
#     mask_ids = []
#     seg_ids = []
#     y = []

#     premise_list = df['sentence1'].to_list()
#     hypothesis_list = df['sentence2'].to_list()
#     label_list = df['gold_label'].to_list()

#     for (premise, hypothesis, label) in zip(premise_list, hypothesis_list, label_list):
#       premise_id = self.tokenizer.encode(premise, add_special_tokens = False)
#       hypothesis_id = self.tokenizer.encode(hypothesis, add_special_tokens = False)
#       pair_token_ids = [self.tokenizer.cls_token_id] + premise_id + [self.tokenizer.sep_token_id] + hypothesis_id + [self.tokenizer.sep_token_id]
#       premise_len = len(premise_id)
#       hypothesis_len = len(hypothesis_id)

#       segment_ids = torch.tensor([0] * (premise_len + 2) + [1] * (hypothesis_len + 1))  # sentence 0 and sentence 1
#       attention_mask_ids = torch.tensor([1] * (premise_len + hypothesis_len + 3))  # mask padded values

#       token_ids.append(torch.tensor(pair_token_ids))
#       seg_ids.append(segment_ids)
#       mask_ids.append(attention_mask_ids)
#       y.append(self.label_dict[label])
    
#     token_ids = pad_sequence(token_ids, batch_first=True)
#     mask_ids = pad_sequence(mask_ids, batch_first=True)
#     seg_ids = pad_sequence(seg_ids, batch_first=True)
#     y = torch.tensor(y)
#     dataset = TensorDataset(token_ids, mask_ids, seg_ids, y)
#     print(len(dataset))
#     return dataset

#   def get_data_loaders(self, batch_size=32, shuffle=True):
#     train_loader = DataLoader(
#       self.train_data,
#       shuffle=shuffle,
#       batch_size=batch_size
#     )

#     val_loader = DataLoader(
#       self.val_data,
#       shuffle=shuffle,
#       batch_size=batch_size
#     )

#     return train_loader, val_loader

In [63]:
# mnli_dataset = MNLIDataBert(multinli_train_reduced, multinli_test_reduced) #for pytorch

392702
9815


In [70]:
# mnli_dataset.train_data[2][3]

tensor(0)

In [75]:
# mnli_dataset.train_data[3]

(tensor([ 101, 2129, 2079, 2017, 2113, 1029, 2035, 2023, 2003, 2037, 2592, 2153,
         1012,  102, 2023, 2592, 7460, 2000, 2068, 1012,  102,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
            0,    0,    0,  

# Preparing for training

In [86]:
label_dict = {'entailment': 0, 'contradiction': 1, 'neutral': 2}
encoding_dict= {"gold_label": {"entailment":0, "contradiction":1,"neutral":2}}
multinli_train_reduced2 = multinli_train_reduced.replace(encoding_dict)
multinli_test_reduced2 = multinli_test_reduced.replace(encoding_dict)



In [91]:
from sklearn.model_selection import train_test_split

multinli_train_reduced2, musltinli_val = train_test_split(multinli_train_reduced2, test_size=0.2)

In [83]:
#Define your train examples. You need more than just two examples...
sen1 = list(multinli_train_reduced2.sentence1)
sen2 = list(multinli_train_reduced2.sentence1)

resulting_list = []
for a,b,label in zip(sen1,sen2,list(multinli_train_reduced2.gold_label)):
    resulting_list.append(InputExample(texts=[a, b], label=label))

In [87]:
#same for val set
#Define your train examples. You need more than just two examples...
sen1 = list(musltinli_val.sentence1)
sen2 = list(musltinli_val.sentence1)

resulting_list_val = []
for a,b,label in zip(sen1,sen2,list(musltinli_val.gold_label)):
    resulting_list_val.append(InputExample(texts=[a, b], label=label))

# Training (Bert+maxpooling+Softmax)

In [90]:
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

    
#Define your train dataset, the dataloader and the train loss
train_dataloader = DataLoader(resulting_list, shuffle=True, batch_size=10)
train_loss = losses.SoftmaxLoss(model,num_labels=3,sentence_embedding_dimension=word_embedding_model.get_word_embedding_dimension())
test_evaluator= evaluation.EmbeddingSimilarityEvaluator.from_input_examples(resulting_list_val, batch_size=10)

#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)],evaluator=test_evaluator, epochs=1, warmup_steps=100, show_progress_bar=True)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Epoch:   0%|          | 0/1 [01:06<?, ?it/s]


KeyboardInterrupt: 

# Testing on the testing set

In [95]:
#Define your test examples
sen1 = list(multinli_test_reduced2.sentence1)
sen2 = list(multinli_test_reduced2.sentence1)

resulting_list_test = []
for a,b,label in zip(sen1,sen2,list(multinli_test_reduced2.gold_label)):
    resulting_list_test.append(InputExample(texts=[a, b], label=label))

In [96]:
from sentence_transformers import SentenceTransformer, InputExample, losses, models, evaluation

evaluator = evaluation.EmbeddingSimilarityEvaluator.from_input_examples(resulting_list_test)

evaluator(model, output_path = "./eval_testset/")

# END IS HERE <<<<<<<<<<<<<<<<<<>>>>>>>>>>>>>>>>>>

In [ ]:
#some code is from the docs, which you might want to find here --> 
#https://www.sbert.net/docs/training/overview.html 

#Define the model. We do the basic bert + mean Pooling
word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


#Define your train dataset, the dataloader and the train loss
train_loss = losses.CosineSimilarityLoss(model)


#Tune the model
model.fit(train_objectives=[(train_dataloader, train_loss)], epochs=1, warmup_steps=100)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [60]:
# #that's actually the sbert one

# from sentence_transformers import SentenceTransformer
# model = SentenceTransformer('paraphrase-MiniLM-L6-v2')

# #Our sentences we like to encode
# sentences = ['This framework generates embeddings for each input sentence',
#     'Sentences are passed as a list of string.',
#     'The quick brown fox jumps over the lazy dog.']

# #Sentences are encoded by calling model.encode()
# embeddings = model.encode(sentences)

# #Print the embeddings
# for sentence, embedding in zip(sentences, embeddings):
#     print("Sentence:", sentence)
#     print("Embedding:", embedding)
#     print("")

Sentence: This framework generates embeddings for each input sentence
Embedding: [-1.76214516e-01  1.20601311e-01 -2.93623805e-01 -2.29858026e-01
 -8.22923705e-02  2.37709388e-01  3.39984953e-01 -7.80964315e-01
  1.18127719e-01  1.63373917e-01 -1.37715399e-01  2.40282431e-01
  4.25125599e-01  1.72417775e-01  1.05279572e-01  5.18164217e-01
  6.22221269e-02  3.99285942e-01 -1.81652486e-01 -5.85578620e-01
  4.49718162e-02 -1.72750562e-01 -2.68443406e-01 -1.47386089e-01
 -1.89217791e-01  1.92150444e-01 -3.83842468e-01 -3.96006793e-01
  4.30648834e-01 -3.15319479e-01  3.65949929e-01  6.05155788e-02
  3.57325852e-01  1.59736261e-01 -3.00983965e-01  2.63250142e-01
 -3.94310951e-01  1.84855536e-01 -3.99549067e-01 -2.67889649e-01
 -5.45117319e-01 -3.13405395e-02 -4.30643976e-01  1.33278191e-01
 -1.74793944e-01 -4.35465515e-01 -4.77378905e-01  7.12556392e-02
 -7.37002045e-02  5.69137037e-01 -2.82579571e-01  5.24973609e-02
 -8.20008039e-01  1.98296845e-01  1.69511810e-01  2.71779805e-01
  2.64610

In [61]:
model

SentenceTransformer(
  (0): Transformer({'max_seq_length': 128, 'do_lower_case': False}) with Transformer model: BertModel 
  (1): Pooling({'word_embedding_dimension': 384, 'pooling_mode_cls_token': False, 'pooling_mode_mean_tokens': True, 'pooling_mode_max_tokens': False, 'pooling_mode_mean_sqrt_len_tokens': False})
)

In [3]:
sts_train=pd.read_csv("stsbenchmark/sts-train.csv",sep='\t',header=None,usecols=[4, 5, 6], quoting=QUOTE_NONE,names=["label","sen1","sen2"])

In [4]:
#Unlike describe in the assignment the similarity is [0,5], not [1,5]
sts_train.describe()

,label
count,5749.000000
mean,2.700999
std,1.464398
min,0.000000
25%,1.500000
50%,3.000000
75%,3.800000
max,5.000000


In [3]:
# from transformers import BertTokenizer, BertModel

# # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # model = BertModel.from_pretrained("bert-base-uncased")
# word_embedding_model = models.Transformer('bert-base-uncased', max_seq_length=256)
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())

# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [56]:
# text = "This fucking thing better works."

# encoded_input = tokenizer(text, return_tensors='pt')
# print(type(encoded_input))
# output = model(**encoded_input)



tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained("bert-base-uncased")
text = ["Dummy example, does this shit even work?","oimfowemfo","Replace me by any text you'd like.", "WTF is this"]#"Dummy example, does this shit even work?"
encoded_input = tokenizer(text, return_tensors='pt', padding=True)
output = model(**encoded_input)
output
print(type(output))

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


<class 'transformers.modeling_outputs.BaseModelOutputWithPoolingAndCrossAttentions'>


In [ ]:
# #This here should hopefully make meaningful embeddings from BERT(not SBERT)

# from transformers import BertTokenizer, BertModel
# # tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# # model = BertModel.from_pretrained("bert-base-uncased")

# tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
# model1 = BertModel.from_pretrained("bert-base-uncased")
# model2 = BertModel.from_pretrained("bert-base-uncased")


# #load datasets
# sts_train=pd.read_csv("stsbenchmark/sts-train.csv",sep='\t',header=None,usecols=[4, 5, 6], quoting=QUOTE_NONE,names=["label","sen1","sen2"])
# sts_test=pd.read_csv("stsbenchmark/sts-test.csv",sep='\t',header=None,usecols=[4, 5, 6], quoting=QUOTE_NONE,names=["label","sen1","sen2"])
# sts_dev=pd.read_csv("stsbenchmark/sts-dev.csv",sep='\t',header=None,usecols=[4, 5, 6], quoting=QUOTE_NONE,names=["label","sen1","sen2"])
# print("loading data ok")
# print("starting tokenizing the data")


# encoded_input1 = tokenizer(list(sts_train.sen1), return_tensors='pt', padding=True)
# output = model1(**encoded_input1)
# encoded_input2 = tokenizer(list(sts_train.sen2), return_tensors='pt', padding=True)
# output2 = model1(**encoded_input2)


# pooling_layer1= models.Pooling(200,pooling_mode_mean_tokens=True)
# pooling_layer2= models.Pooling(200,pooling_mode_mean_tokens=True)


# final_model1= SentenceTransformer(modules=[model1,pooling_layer1])
# embedding1=final_model1.encode(encoded_input1, batch_size=128, convert_to_numpy=True, show_progress_bar=True)


# final_model2= SentenceTransformer(modules=[model2,pooling_layer2])
# embedding2=final_model2.encode(encoded_input2, batch_size=128, convert_to_numpy=True, show_progress_bar=True)


# text = ['This fucking thing better works',"Hate this bla bla, what the fuck is this for"]
# encoded_input = tokenizer(text, return_tensors='pt', padding=True)
# output = final_model1(**encoded_input)

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.decoder.weight', 'cls.predictions.tr

loading data ok
starting tokenizing the data


In [170]:
# train_sen1_encoding= tokenizer(list(sts_train["sen1"]), padding="max_length", truncation=True)
# train_sen2_encoding= tokenizer(list(sts_train["sen2"]), padding="max_length", truncation=True)
# sts_train.head(5) # we split the process up here so you can see the difference on the label

,label,sen1,sen2
0,5.00,A plane is taking off.,An air plane is taking off.
1,3.80,A man is playing a large flute.,A man is playing a flute.
2,3.80,A man is spreading shreded cheese on a pizza.,A man is spreading shredded cheese on an uncoo...
3,2.60,Three men are playing chess.,Two men are playing chess.
4,4.25,A man is playing the cello.,A man seated is playing the cello.
